# Promedio Diario $PM_{10}$

Para nuestra transformación de datos, utilizaremos las siguientes librerias:



In [88]:
import pandas as pd
import numpy as np

- Cargando dataset a un DataFrame de pandas

- Posteriormente normalizando la columna `Fecha` a tipo datetime

In [89]:
#Cargando archivos
df = pd.read_csv('https://raw.githubusercontent.com/IrvingC48/BeduFase2-Proyecto_python/main/Datasets/PM10/PM10_depurado.csv')

# Normalizando fechas
df['FECHA'] = pd.to_datetime(df['FECHA'], unit='ns')
df.head()

,Unnamed: 0,FECHA,HORA,ACO,AJM,ATI,BJU,CAM,CUA,CUT,FAC,GAM,HGM,INN,IZT,MER,PED,SAG,SFE,TAH,TLA,UIZ,VIF,XAL
0,0,2019-01-01,1,201.0,39.0,105.0,84.0,120.0,113.0,149.0,78.0,NaN,77.0,31.0,132.0,113.0,NaN,149.0,50.0,191.0,78.0,129.0,280.0,155.0
1,1,2019-01-01,2,218.0,27.0,128.0,115.0,135.0,158.0,211.0,111.0,NaN,81.0,20.0,202.0,141.0,NaN,149.0,54.0,225.0,73.0,125.0,257.0,236.0
2,2,2019-01-01,3,240.0,20.0,93.0,134.0,177.0,192.0,214.0,137.0,NaN,96.0,12.0,198.0,118.0,NaN,152.0,65.0,185.0,80.0,132.0,509.0,337.0
3,3,2019-01-01,4,231.0,10.0,86.0,124.0,204.0,143.0,178.0,161.0,NaN,112.0,12.0,192.0,154.0,NaN,217.0,33.0,172.0,84.0,136.0,611.0,271.0
4,4,2019-01-01,5,216.0,7.0,68.0,143.0,162.0,123.0,234.0,103.0,NaN,110.0,5.0,178.0,227.0,NaN,242.0,29.0,83.0,63.0,191.0,653.0,226.0


- Reacomodo del DataFrame , donde ahora tendremos una columna para las estaciones y sus correspondientes datos de medición
- Eliminando los datos `NaN`
- Comprobando que no queden datos `NaN` en el DataFrame

In [90]:
# Extraer nombre de columnas
cols = df.columns.tolist()

# Reacomodando dataframe
df = pd.melt(df, id_vars=['FECHA', 'HORA'], value_vars=cols[3:], var_name='station', value_name='measurement')

# df = df[~df['measurement'].isna()] otra forma de eliminar nan's
df = df.dropna(axis=0, how='any').reset_index(drop=True)

print("No. de columnas que contienen valores nulos ")
print(len(df.columns[df.isna().any()]))

print("No. de columnas que no contienen valores nulos")
print(len(df.columns[df.notna().all()]))

print("No total de columnas en el dataframe")
print(len(df.columns))
df.head()

No. de columnas que contienen valores nulos 
0
No. de columnas que no contienen valores nulos
4
No total de columnas en el dataframe
4


,FECHA,HORA,station,measurement
0,2019-01-01,1,ACO,201.0
1,2019-01-01,2,ACO,218.0
2,2019-01-01,3,ACO,240.0
3,2019-01-01,4,ACO,231.0
4,2019-01-01,5,ACO,216.0


- Ordenando el dataframe segun la fecha y la estación

In [91]:
# ordenando el dataframe por fecha y nombre de estacion
df = df.sort_values(['FECHA', 'station']).reset_index(drop=True)
df

,FECHA,HORA,station,measurement
0,2019-01-01,1,ACO,201.0
1,2019-01-01,2,ACO,218.0
2,2019-01-01,3,ACO,240.0
3,2019-01-01,4,ACO,231.0
4,2019-01-01,5,ACO,216.0
...,...,...,...,...
259447,2020-12-31,20,VIF,54.0
259448,2020-12-31,21,VIF,52.0
259449,2020-12-31,22,VIF,52.0
259450,2020-12-31,23,VIF,178.0


Ahora se busca eliminar, según el día y la estación, aquellas filas que tengan mediciones horarias menores a las requeridas por la normativa (18 mediciones). Por ejemplo, si la estación ACO en el 01 de enero de 2019 registró solamente 15 mediciones horarias, todas las mediciones de ese día para la estación ACO no será tomadas en cuenta para los posteriores análisis. 

Primero se debe elaborar una función que devuelva el dataframe que contenga solamente los registros válidos

In [92]:
def dia_estacion_valido(dataframe):
  #Contar las mediciones horarias por día y estación
  filtro= dataframe.groupby(['FECHA', 'station'],as_index=False).agg(N_h=('HORA','count'))
  #Filtrar aquellas que no cumplen normativa
  filtro= filtro.query('N_h < 17')
  #Conservar las columnas de fecha y estación
  filtro=filtro.loc[:,['FECHA','station']]
  #Eliminar del dataframe original las mediciones que no cumplen normativa
  filtro=pd.merge(dataframe,filtro, how='outer', indicator=True)
  filtro=filtro[filtro['_merge'] == 'left_only']
  return filtro


Con la función, se filtra las mediciones por día y por estación son válidas para los siguientes análisis.

In [97]:

df_filtrado= dia_estacion_valido(df)
df_filtrado = df_filtrado.drop(columns=['_merge'])
df_filtrado=df_filtrado.reset_index()
df_filtrado


,index,FECHA,HORA,station,measurement
0,0,2019-01-01,1,ACO,201.0
1,1,2019-01-01,2,ACO,218.0
2,2,2019-01-01,3,ACO,240.0
3,3,2019-01-01,4,ACO,231.0
4,4,2019-01-01,5,ACO,216.0
...,...,...,...,...,...
253977,259447,2020-12-31,20,VIF,54.0
253978,259448,2020-12-31,21,VIF,52.0
253979,259449,2020-12-31,22,VIF,52.0
253980,259450,2020-12-31,23,VIF,178.0


Una vez con el dataframe inicial limpio, se procede a relacionar cada estación con su respectiva zona de activación. Primeramente se leen los datos

In [98]:
# Cargando dataset de zonas
df_zonas = pd.read_csv('https://raw.githubusercontent.com/IrvingC48/BeduFase2-Proyecto_python/main/Datasets/cat_estacion_depurado.csv')

# Eliminando columna 'Unnamed: 0'
df_zonas = df_zonas.drop(columns=['Unnamed: 0'])
# Remplazando los - por NaN
df_zonas = df_zonas.replace('-', np.nan)
# Eliminando lo NaN's
df_zonas = df_zonas.dropna(axis=0, how='any').reset_index(drop=True)
# Renombrando la columna
df_zonas = df_zonas.rename(columns={'cve_estac': 'station'})
df_zonas.head()

,station,nom_estac,Zona
0,ACO,Acolman,NE
1,AJU,Ajusco,SO
2,AJM,Ajusco Medio,SO
3,ATI,Atizapan,NO
4,BJU,Benito Ju�rez,CE


- Uniendo los datos con las zonas correspondiente


In [99]:
df_filtrado = df_filtrado.merge(df_zonas[['station', 'Zona']], on='station')
df_filtrado.head()

,index,FECHA,HORA,station,measurement,Zona
0,0,2019-01-01,1,ACO,201.0,NE
1,1,2019-01-01,2,ACO,218.0,NE
2,2,2019-01-01,3,ACO,240.0,NE
3,3,2019-01-01,4,ACO,231.0,NE
4,4,2019-01-01,5,ACO,216.0,NE


Con los datos válidos para realizar las operaciones siguientes y ya con la relación de cada estación con su respectiva zona de activación, se procede a calcular el promedio diario de concentración y el índice de calidad del aire en cada zona y en cada día

### Calculando el promedio diario
Para el cálculo del promedio diario de $PM_{10}$ se utilizó la metodología descrita en la `NOM-025-SSA1-2014`, siguiendo la siguiente ecuación:

$$\bar{x}= \frac{1}{n} \displaystyle\sum_{i=1}^n x_i$$
$\bar{x}:$ promedio de 24 horas, 

$n:$ número de concentraciones horarias válidas

$x_i:$ concentraciones horarias válidas

- Camo datos extra obteniendo los valores mínimos y máximos


In [100]:
# Calculando promedio diario, contando numero de horas usadas, obteniendo el maximo y minimo para las visualizaciones en el módulo siguiente
df_PM1024h = df_filtrado.groupby(['FECHA', 'Zona'], as_index=False).agg(
            PromDiario=('measurement', lambda x: round(x.mean(),0)), 
            Max = ('measurement', 'max'),
            Min = ('measurement', 'min'))
df_PM1024h.head()

,FECHA,Zona,PromDiario,Max,Min
0,2019-01-01,CE,97.0,227.0,21.0
1,2019-01-01,NE,150.0,653.0,20.0
2,2019-01-01,NO,84.0,240.0,2.0
3,2019-01-01,SE,97.0,225.0,3.0
4,2019-01-01,SO,44.0,192.0,4.0


### Calculando Índice de calidad del aire

Donde:
$$\text{Índice}= (k \times (C - BP_{Lo}))+I_{Lo}$$
$$k= \frac{I_{Hi}-I_{Lo}}{BP_{Hi}-BP_{Lo}}$$
 
$\text{Índice}:$ Valor del índice para el contaminante deseado.<br>
$C:$ valor redondeado para la concentración del contaminante.<br>
$k:$ constante de proporcionalidad estimada.<br>
$BP_{Hi}:$ valor del punto de corte que es mayor o igual a la concentración a evaluar.<br>
$BP_{Lo}:$ valor del punto de corte que es menor o igual a la concentración a evaluar.<br>
$I_{Hi}:$ valor del índice que corresponde al valor de $BP_{Hi}$<br>
$I_{Lo}:$ valor del índice que corresponde al valor de $BP_{Lo}$<br>


Primero se crea una función que determine el índice de calidad del aire según la metodología y los valores de la tabla descrita en el apartado Metodología para los valores de los puntos de corte

In [101]:
def AQI(df):
    CP = df['PromDiario']
    i = 0

    if 0 <= CP <= 40:
        k = (50-0)/(40-0)
        i = k * (CP - 0) + 0
    if 41 <= CP <= 75:
        k = (100-51)/(75-41)
        i = k * (CP - 41) + 51
    if 76 <= CP <= 214:
        k = (150-101)/(214-76)
        i = k * (CP - 76) + 101
    if 215 <= CP <= 354:
        k = (200-151)/(354-215)
        i = k * (CP - 215) + 151
    if 355 <= CP <= 424:
        k = (300-201)/(242-355)
        i = k * (CP - 355) + 201
    if 425 <= CP <= 504:
        k = (400-301)/(504-425)
        i = k * (CP - 425) + 301
    if 505 <= CP <= 604:
        k = (500-401)/(604-505)
        i = k * (CP - 505) + 401
    
    return round(i, 3)

Luego se aplica la función para cada promedio diario calculado



In [102]:
df_PM1024h['indice'] = df_PM1024h.apply(AQI, axis=1)
df_PM1024h.head()

,FECHA,Zona,PromDiario,Max,Min,indice
0,2019-01-01,CE,97.0,227.0,21.0,108.457
1,2019-01-01,NE,150.0,653.0,20.0,127.275
2,2019-01-01,NO,84.0,240.0,2.0,103.841
3,2019-01-01,SE,97.0,225.0,3.0,108.457
4,2019-01-01,SO,44.0,192.0,4.0,55.324


Con el índice de calidad del aire, se crea otra función para determinar la categoría de calidad del aire descrita por la norma mexicana.

In [85]:
def AQI_categoria(df):
    indice = df['indice']
    if indice <= 50:
      clase='Buena'
    elif indice <= 100:
      clase='Regular'
    elif indice <= 150:
      clase= 'Mala'
    elif indice <= '200':
      clase = 'Muy mala'
    else: 
      clase = 'Extremadamente mala'
    
    return clase

Se aplica la función para determinar la categoría a cada promedio diario calculado.

In [103]:
df_PM1024h['clase'] = df_PM1024h.apply(AQI_categoria, axis=1)
df_PM1024h.head()

,FECHA,Zona,PromDiario,Max,Min,indice,clase
0,2019-01-01,CE,97.0,227.0,21.0,108.457,Mala
1,2019-01-01,NE,150.0,653.0,20.0,127.275,Mala
2,2019-01-01,NO,84.0,240.0,2.0,103.841,Mala
3,2019-01-01,SE,97.0,225.0,3.0,108.457,Mala
4,2019-01-01,SO,44.0,192.0,4.0,55.324,Regular


Se guardan los resultados en un csv

In [ ]:
data_PM1024h.to_csv('../Datasets/PM10/PDEstacion_PM10.csv', index=False)